In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Figure 4: Cluster Profiles

In [ ]:
color_palette= ['#332288',  # dark blue
                          '#117733',  # green
                          '#44AA99',  # teal
                          '#88CCEE',  # light blue
                          '#DDCC77',  # yellow
                          '#CC6677',  # red
                          '#AA4499',  # purple
                          '#661100',  # moved dark brown
                          '#888888',  # moved medium gray
                          '#999933',  # olive
                          '#882255',  # moved brown
                          '#6699CC']  # steel blue

#load feature array
feat_array = pd.read_csv('Data/feature_array_daily_access.csv').values
timesteps = int(np.shape(feat_array)[1])

In [ ]:
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'

num_clusters = 15
fig_id = ['a.', 'b.', 'c.', 'd.', 'e.', 'f.', 'g.', 'h.', 'i.', 'j.', 'k.',  'l.', 'm.', 'n.', 'o.']

#set up figure
fig = plt.figure(figsize=(14, 18))
ax = []
for i in range(1, num_clusters+1):
    ax.append(fig.add_subplot(6, 3, i))
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.3)

#read cluster labels and centers
cluster_labels = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_labels_'+str(num_clusters)+'.csv', header=None)
cluster_centers = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_centers_'+str(num_clusters)+'.csv', header=None)

unique_clusters = np.unique(cluster_labels.values)
cluster_info_df = pd.DataFrame(columns=['Orig Cluster Label', 'Area Under Curve', 'Sorted Label'])
cluster_info_df['Orig Cluster Label'] = unique_clusters

for c in range(num_clusters):
    #calculate area under curve for each cluster
    cluster_info_df.loc[c, 'Area Under Curve'] = np.trapz(cluster_centers.iloc[c, :].values)

#sort clusters by area under curve and make a new column that saves the old index values from 0-17
cluster_info_df['old index'] = cluster_info_df.index
cluster_info_df = cluster_info_df.sort_values(by='Area Under Curve', ascending=False)
cluster_info_df['Sorted Label'] = np.arange(1, num_clusters+1)
cluster_info_df.to_csv('Data/Cluster_Results_Adaptive/cluster_info_'+str(num_clusters)+'.csv', index=False)

for c in range(num_clusters):
    #plot charger availability curve for each driver from the feature array
    for i, driver in enumerate(np.where(cluster_labels==cluster_info_df['Orig Cluster Label'].values[c])[0]):
        ax[c].plot(range(timesteps),feat_array[driver, :], color = color_palette[int(c%12)], alpha=0.2)
        
    #plot cluster center
    ax[c].plot(range(timesteps),cluster_centers.iloc[cluster_info_df['old index'].values[c], :], linewidth=2.5, color = 'k', label = 'Cluster Center')
    
    print('Cluster ', c+1, 'has ', len(np.where(cluster_labels==cluster_info_df['Orig Cluster Label'].values[c])[0]), 'vehicles')
    ax[c].set_ylim([0,1])
    ax[c].set_xlabel('Time [hours]', fontsize=12)
    ax[c].set_ylabel('Charging Availability', fontsize=12)
    ax[c].set_xticks(np.arange(0, timesteps+1, 60 * 4), labels = np.arange(0, int(timesteps/60)+1, 4))
    ax[c].tick_params(axis='both', which='major', labelsize=12)
    ax[c].legend(title = 'Cluster '+ str(c+1), fontsize=10, title_fontsize=11)
    ax[c].annotate(fig_id[c], xy=(-0.2, 1.07), xycoords='axes fraction', fontsize=14)


plt.savefig(save_path + '/4_cluster_profiles.pdf', bbox_inches='tight')
plt.show()